# Исследование надежности заемщиков


In [2]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [7]:
data['days_employed'] = data['days_employed'].abs()

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [15]:
data['education'] = data['education'].str.lower()

In [16]:
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

### Категоризация данных

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [62]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
data_pivot = data.pivot_table(index = ['children'], values = 'debt', aggfunc = ('sum', 'count', 'mean'))
data_pivot['count'] = data_pivot['count'].astype(int)
data_pivot['sum'] = data_pivot['sum'].astype(int)
data_pivot['mean'] = data_pivot['mean']*100
data_pivot = data_pivot.sort_values(by = 'count', ascending = False)
data_pivot = data_pivot.rename(columns = {'mean' : 'Среднее значение просрочки выплаты', 'count' : 'Количество заемщиков', 'sum' : 'Количество просрочек по кредиту'})
data_pivot.index.names = ['Количество детей']
data_pivot

,Количество заемщиков,Среднее значение просрочки выплаты,Количество просрочек по кредиту
Количество детей,,,
0,14091,7.543822,1063
1,4808,9.234609,444
2,2052,9.454191,194
3,330,8.181818,27
4,41,9.756098,4
5,9,0.000000,0


**Вывод:** 
Можно сделать вывод о том, что между количеством детей и возвратом кредита в срок существует зависимость. Чем больше детей в семье, тем больше задолженностей по возрату кредитов, и наоборот, чем меньше детей в семье, тем меньше задолженностей по возрату кредитов. Люди, которые не имеют детей, лучше всего отдают кредит, доля должников здесь составляет с 7,5%, хуже обстоит дело с кредиторами, кто имеет одного или двух детей, здесь доля невозврата составляет соответственно 9,2% и 9,4%. Остальные категории уже некорректно сравнивать, так как в них очень маленькая выборка.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [56]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
data_pivot = data.pivot_table(index = ['family_status'], values = 'debt', aggfunc = ('sum', 'count', 'mean'))
data_pivot['count'] = data_pivot['count'].astype(int)
data_pivot['sum'] = data_pivot['sum'].astype(int)
data_pivot['mean'] = data_pivot['mean']*100
data_pivot = data_pivot.sort_values(by = 'count', ascending = False)
data_pivot = data_pivot.rename(columns = {'mean' : 'Среднее значение просрочки выплаты', 'count' : 'Количество заемщиков', 'sum' : 'Количество просрочек по кредиту'})
data_pivot.index.names = ['Семейный статус']
data_pivot

,Количество заемщиков,Среднее значение просрочки выплаты,Количество просрочек по кредиту
Семейный статус,,,
женат / замужем,12261,7.560558,927
гражданский брак,4134,9.313014,385
Не женат / не замужем,2796,9.763948,273
в разводе,1189,7.064760,84
вдовец / вдова,951,6.624606,63


**Вывод:** 
Можно сделать вывод о том, что семейный статус влияет на возрат кретитов. Люди, которые в разводе и имеющие семейный статус вдовец / вдова, лучше всего отдают кредит, доля должников здесь составляет 6,6%-7%, хуже обстоит дело с кредиторами, кто женат / замужем, здесь доля невозврата составляет соответственно 7,5%. Больший процент невозрата у клинтов, имеющих семейный статус не женат / не замужем, а также находящихся в гражданском браке, у данных категорий процент невозрата кредита составляет 9.3% - 9,8%. 

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [61]:
def get_income_group(income):
    if income <= 50000:
        return "Очень низкий"
    elif income <= 150000:
        return "Низкий"
    elif income <= 250000:
        return "Средний"
    elif income <= 500000:
        return "Высокий"
    return "Очень высокий"
data['category_income'] = data['total_income'].apply(get_income_group)
data_pivot = data.pivot_table(index = ['category_income'], values = 'debt', aggfunc = ('count', 'mean', 'sum'))
data_pivot['count'] = data_pivot['count'].astype(int)
data_pivot['sum'] = data_pivot['sum'].astype(int)
data_pivot['mean'] = data_pivot['mean']*100
data_pivot = data_pivot.sort_values(by = 'mean', ascending = False)
data_pivot = data_pivot.rename(columns = {'mean' : 'Среднее значение просрочки выплаты', 'count' : 'Количество заемщиков', 'sum' : 'Количество просрочек по кредиту'})
data_pivot.index.names = ['Уровень дохода']
data_pivot

,Количество заемщиков,Среднее значение просрочки выплаты,Количество просрочек по кредиту
Уровень дохода,,,
Низкий,11183,8.495037,950
Средний,6980,8.094556,565
Высокий,2575,6.990291,180
Очень высокий,222,6.306306,14
Очень низкий,371,6.199461,23


**Вывод:** 
Разбив ежемесячный доход на категории, можно сделать следующий вывод:
Меньше всего задолженностей имеют клиенты с высокий уровень дохода (От 250 тыс. рублей до 500 тыс. рублей), невозрат кредита в срок составляет 7%.
Хуже обстоит дело с клиентами, имеющими средний уровень дохода (от 150 тыс. до 250 тыс. рублей), т.к просрочка выплаты составляет 8%
Больше всего задолженностей имеют клиенты с низким уровнем дохода (от 50 тыс. до 150 тыс. рублей), т.к просрочка выплаты составляет 8.5%.
Выборка клиентов с очень низким (до 50 тыс. рублей) и очень высоким уровнем дохода (от 500 тыс. рублей) ялвляется недостоверной из-за недостаточного количества данных.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [63]:
data_pivot = data.pivot_table(index = ['purpose_category'], values = 'debt', aggfunc = ('sum', 'count', 'mean'))
data_pivot['count'] = data_pivot['count'].astype(int)
data_pivot['sum'] = data_pivot['sum'].astype(int)
data_pivot['mean'] = data_pivot['mean']*100
data_pivot = data_pivot.sort_values(by = 'count', ascending = False)
data_pivot = data_pivot.rename(columns = {'mean' : 'Среднее значение просрочки выплаты', 'count' : 'Количество заемщиков', 'sum' : 'Количество просрочек по кредиту'})
data_pivot.index.names = ['Цель кредита']
data_pivot

,Количество заемщиков,Среднее значение просрочки выплаты,Количество просрочек по кредиту
Цель кредита,,,
операции с недвижимостью,10751,7.255139,780
операции с автомобилем,4279,9.347978,400
получение образования,3988,9.252758,369
проведение свадьбы,2313,7.911803,183


**Вывод:** 
Меньше всего задолженностей имеют клиенты, совершившие операции с недвижимостью, т.к. невозврат в срок составляет 7.2%. Доля просрочки выплат по кредиту, взятому на проведение свадьбы, составляет 7,9%. 
Хуже обстоит дело с кредиторами, взявшими кредит на получение образования и операции с автомобилем, здесь доля невозврата составляет соответственно 9,2% и 9,3%.


#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* 
Пропуски в данных появляются разными путями. Например, пользователь не указал о себе какие-нибудь сведения или автоматизированная система сбора информации дала сбой. Иногда пропуски оставляют умышленно, рассчитывая на автозаполнение спецсимволами.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* 
Пропуски в количественных переменных заполняют характерными значениями. Это значения, характеризующие состояние выборки, — набора данных, выбранных для проведения исследования. Чтобы примерно оценить типичные значения выборки, можно использовать среднее арифметическое или медиану, но среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства. Поэтому медиана дает более объективную информацию.

### Шаг 4: общий вывод.

Используя входные данные от банка — статистику о платёжеспособности клиентов для построения модели кредитного скоринга, были сделаны следующие выводы: 
- между количеством детей и возвратом кредита в срок существует зависимость. Чем больше детей в семье, тем больше задолженностей по возрату кредитов, и наоборот, чем меньше детей в семье, тем меньше задолженностей по возрату кредитов. Люди, которые не имеют детей, лучше всего отдают кредит, доля должников здесь составляет с 7,5%, хуже обстоит дело с кредиторами, кто имеет одного или двух детей, здесь доля невозврата составляет соответственно 9,2% и 9,4%. Остальные категории уже некорректно сравнивать, так как в них очень маленькая выборка;

- семейный статус влияет на возрат кретитов. Люди, которые в разводе и имеющие семейный статус вдовец / вдова, лучше всего отдают кредит, доля должников здесь составляет 6,6%-7%, хуже обстоит дело с кредиторами, кто женат / замужем, здесь доля невозврата составляет соответственно 7,5%. Больший процент невозрата у клинтов, имеющих семейный статус не женат / не замужем, а также находящихся в гражданском браке, у данных категорий процент невозрата кредита составляет 9.3% - 9,8%;

- Разбив ежемесячный доход на категории, можно сделать следующий вывод:
Меньше всего задолженностей имеют клиенты с высокий уровень дохода (От 250 тыс. рублей до 500 тыс. рублей), невозрат кредита в срок составляет 7%.
Хуже обстоит дело с клиентами, имеющими средний уровень дохода (от 150 тыс. до 250 тыс. рублей), т.к просрочка выплаты составляет 8%
Больше всего задолженностей имеют клиенты с низким уровнем дохода (от 50 тыс. до 150 тыс. рублей), т.к просрочка выплаты составляет 8.5%.
Выборка клиентов с очень низким (до 50 тыс. рублей) и очень высоким уровнем дохода (от 500 тыс. рублей) ялвляется недостоверной из-за недостаточного количества данных;

- Меньше всего задолженностей имеют клиенты, совершившие операции с недвижимостью, т.к. невозврат в срок составляет 7.2%. Доля просрочки выплат по кредиту, взятому на проведение свадьбы, составляет 7,9%. Хуже обстоит дело с кредиторами, взявшими кредит на получение образования и операции с автомобилем, здесь доля невозврата составляет соответственно 9,2% и 9,3%.

Анализируя полученную статистику, можно выявить наиболее платежеспособного клиента. Им является вдовец / вдова, не имеющий детей, с высоким ежемесячным доходом (От 250 тыс. рублей до 500 тыс. рублей), взявший кредит на совершение операций с недвижимостью. Наименьшим платежеспособным клиентом является человек, имеющий семейный статус не женат / не замужем, имеющий одного или двух детей, с низким уровнем дохода (от 50 тыс. до 150 тыс. рублей) и взявщий кредит на операции с автомобилем.